In [1]:
import os

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision.datasets
import torchvision.transforms as transforms
import torchvision

from mean_teacher import datasets, architectures

from IPython.display import clear_output

device = "cuda" if torch.cuda.is_available() else "cpu"

import matplotlib.pyplot as plt

to_image = transforms.ToPILImage()

In [2]:
dataset_config = datasets.__dict__['sslMini']()

In [3]:
def load_weights(model_arch, pretrained_model_path, state_dict, cuda=True):
        # Load pretrained model
        pretrained_model = torch.load(f=pretrained_model_path, map_location="cuda" if cuda else "cpu")

        from collections import OrderedDict
        new_state_dict = OrderedDict()
        for k, v in pretrained_model[state_dict].items():
            name = k[7:] # remove `module.`
            new_state_dict[name] = v

        # Load pre-trained weights in current model
        with torch.no_grad():
            model_arch.load_state_dict(new_state_dict, strict=True)

        # Debug loading
        #print('Parameters found in pretrained model:')
        pretrained_layers = new_state_dict.keys()
        #for l in pretrained_layers:
        #    print('\t' + l)
        #print('')

        for name, module in model_arch.state_dict().items():
            if name in pretrained_layers:
                assert torch.equal(new_state_dict[name].cpu(), module.cpu())
                #print('{} have been loaded correctly in current model.'.format(name))
            else:
                raise ValueError("state_dict() keys do not match")
                
        return model_arch


In [5]:
BATCH_SIZE = 64

evaldir = "/scratch/ijh216/ssl_mini/supervised/val"

eval_loader = torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(evaldir, dataset_config['eval_transformation']),
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              num_workers=2,
                                              #pin_memory=True,
                                              drop_last=False)


In [62]:
model_dir = "/scratch/ijh216/ssl/ssl_shake_mini_augment/2019-05-06_18-04-18/10/transient/checkpoint.325.ckpt" 
model = architectures.__dict__['cifar_shakeshake26']().to(device)
model = load_weights(model, model_dir, state_dict="ema_state_dict", cuda=True)

In [41]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    labeled_minibatch_size = max(target.ne(-1).sum(), 1e-8).item()

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True).item()
        print(correct_k)
        print(correct_k * (100.0 / labeled_minibatch_size))
        res.append(correct_k * (100.0 / labeled_minibatch_size))
    return res

In [63]:
n_samples = 0.
n_correct_top_1 = 0
n_correct_top_k = 0

for i, (img, target) in enumerate(eval_loader):
    img, target = img.to(device), target.to(device)
    n_samples += BATCH_SIZE

        # Forward
    output = model(img)[0]

        # Top 1 accuracy
    pred_top_1 = torch.topk(output, k=1, dim=1)[1]
    n_correct_top_1 += pred_top_1.eq(target.view_as(pred_top_1)).int().sum().item()

        # Top k accuracy
    pred_top_k = torch.topk(output, k=5, dim=1)[1]
    target_top_k = target.view(-1, 1).expand(BATCH_SIZE, 5)
    n_correct_top_k += pred_top_k.eq(target_top_k).int().sum().item()
    
    if i % 100 == 0:
        print("******************************")
        print("Acc@1", n_correct_top_1/n_samples)
        print("Acc@5", n_correct_top_k/n_samples)
        print("******************************")
    
    # Accuracy
top_1_acc = n_correct_top_1/n_samples
top_k_acc = n_correct_top_k/n_samples

print("******************************")
print("Acc@1", top_1_acc)
print("Acc@5", top_k_acc)
print("******************************")

******************************
Acc@1 0.265625
Acc@5 0.5
******************************
******************************
Acc@1 0.30089727722772275
Acc@5 0.536355198019802
******************************
******************************
Acc@1 0.3004508706467662
Acc@5 0.5382462686567164
******************************
******************************
Acc@1 0.30424626245847175
Acc@5 0.5367005813953488
******************************
******************************
Acc@1 0.3063045511221945
Acc@5 0.5393157730673317
******************************
******************************
Acc@1 0.3060625
Acc@5 0.539625
******************************


In [61]:
n_samples = 0.
n_correct_top_1 = 0
n_correct_top_k = 0

for i, (img, target) in enumerate(eval_loader):
    img, target = img.to(device), target.to(device)
    n_samples += BATCH_SIZE

        # Forward
    output = model(img)[0]

        # Top 1 accuracy
    pred_top_1 = torch.topk(output, k=1, dim=1)[1]
    n_correct_top_1 += pred_top_1.eq(target.view_as(pred_top_1)).int().sum().item()

        # Top k accuracy
    pred_top_k = torch.topk(output, k=5, dim=1)[1]
    target_top_k = target.view(-1, 1).expand(BATCH_SIZE, 5)
    n_correct_top_k += pred_top_k.eq(target_top_k).int().sum().item()
    
    if i % 100 == 0:
        print("******************************")
        print("Acc@1", n_correct_top_1/n_samples)
        print("Acc@5", n_correct_top_k/n_samples)
        print("******************************")
    
    # Accuracy
top_1_acc = n_correct_top_1/n_samples
top_k_acc = n_correct_top_k/n_samples

print("******************************")
print("Acc@1", top_1_acc)
print("Acc@5", top_k_acc)
print("******************************")

******************************
Acc@1 0.125
Acc@5 0.234375
******************************
******************************
Acc@1 0.1639851485148515
Acc@5 0.3103341584158416
******************************
******************************
Acc@1 0.16752176616915423
Acc@5 0.31467661691542287
******************************
******************************
Acc@1 0.16715116279069767
Acc@5 0.3136939368770764
******************************
******************************
Acc@1 0.1684071072319202
Acc@5 0.31495480049875313
******************************
******************************
Acc@1 0.16746875
Acc@5 0.31615625
******************************
